In [ ]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import backend as K
import random
import os

# Path for the images. Check ReadMe to download image data.
a_dir = os.path.join('rstrain_A_N/A')
b_dir = os.path.join('rstrain_A_N/B')
c_dir = os.path.join('rstrain_A_N/W')

print('Total training A images:', len(os.listdir(a_dir)))
print('Total training B images:', len(os.listdir(b_dir)))
print('Total training W images:', len(os.listdir(c_dir)))

a_files = os.listdir(a_dir)
#print(a_files[:10])

b_files = os.listdir(b_dir)
#print(b_files[:10])

c_files = os.listdir(c_dir)
#print(c_files[:10])

img_width, img_height = 160, 120 # size of images


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

#Defining image data directories and 

TRAINING_DIR = "rstrain_A_N"
training_datagen = ImageDataGenerator(rescale = 1./255)

VALIDATION_DIR = "rsvalidation_A_N"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(120,160),
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(120,160),
    class_mode='categorical')

# ...
from tensorflow.keras import regularizers
model = tf.keras.models.Sequential([
    # First convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', 
                           kernel_regularizer=regularizers.l2(0.001),
                           input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', 
                           kernel_regularizer=regularizers.l2(0.001)
                           ),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The Third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu',
                           kernel_regularizer=regularizers.l2(0.001)
                           ),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu',
                           kernel_regularizer=regularizers.l2(0.001)
                           ),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Flatten and Dropout
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    
    # 128 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=10E-7)

# Stop based on minimum validation loss with 10 step patience.
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# Save checkpoint based on maximum validation accuracy (edit for appropriate path).
from tensorflow.keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('/path/Mod_name.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

history = model.fit(train_generator, epochs=300, batch_size=64, validation_data = validation_generator, verbose = 1, callbacks=[es,mc])